In [8]:
import psycopg2
import pandas as pd

In [2]:
connection = psycopg2.connect(user = "machineteaching",
                                  password = "***REMOVED***",
                                  host = "localhost",
#                                   port = "5432",
                                  database = "machineteaching")
connection.autocommit=True
cursor = connection.cursor()

In [53]:
create_table_query = '''CREATE TABLE EXPERIMENTS
    (EXPERIMENT_ID SERIAL PRIMARY KEY,
    DATASET TEXT NOT NULL,
    X REAL ARRAY NOT NULL,
    Y REAL ARRAY NOT NULL,
    VECTORIZER TEXT NOT NULL,
    IS_BINARY BOOLEAN NOT NULL,
    MIN_DF REAL NOT NULL,
    DISTANCE TEXT NOT NULL,
    K INTEGER NOT NULL,
    METHOD TEXT NOT NULL,
    MODEL  bytea NOT NULL,
    CLUSTERING_TIME INTEGER NOT NULL,
    GAP REAL NOT NULL,
    GAP_STD REAL NOT NULL,
    GAP_TIME REAL NOT NULL,
    SILHOUETTE REAL NOT NULL,
    SILHOUETTE_SAMPLES REAL ARRAY NOT NULL,
    SILHOUETTE_TIME REAL NOT NULL,
    COHERENCE_SAMPLES REAL ARRAY NOT NULL,
    COHERENCE_MED REAL NOT NULL,
    COHERENCE_STD REAL NOT NULL,
    COHERENCE_TIME REAL NOT NULL,
    COHERENCE_K INTEGER NOT NULL)
    '''

cursor.execute(create_table_query)
connection.commit()

In [54]:
cols = ['method', 'clustering_time', 'gap_time', 'silhouette_time', 'coherence_time']
select_query = '''SELECT %s FROM EXPERIMENTS''' % ", ".join(cols)
cursor.execute(select_query)
r = cursor.fetchall()
print(r)

[]


In [48]:
df = pd.DataFrame(r, columns=cols)

In [49]:
df['total_time'] = df.sum(axis=1)

In [50]:
df

,method,clustering_time,gap_time,silhouette_time,coherence_time,total_time
0,nmf,1,353.823,0.102683,3.68096,358.606643
1,nmf,2,350.142,0.468195,3.35627,355.966465
2,nmf,1,351.739,0.371906,5.14118,358.252086
3,nmf,2,353.816,0.727259,4.99744,361.540699


In [52]:
drop_query = '''DROP TABLE EXPERIMENTS'''
cursor.execute(drop_query)
connection.commit()